# Model Notebook 

## Business & Data Understanding
#### Revisiting our end goals with sombe EDA knowledge
- Stakeholder is a real estate agency.
- We want to create a tool for a real estate agency to estimate sales or purchase prices given housing info.
- This can be done with a regression model.

## Loading packages, libraries, functions and variables from the EDA notebook.

In [1]:
#Loading the needed packages, libraries, functions and variables from the EDA notebook.
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Original DataFrame
%store -r df_original

In [3]:
#Cleaned DataFrame — from the EDA notebook
%store -r df_clean

In [4]:
from haversine import haversine, Unit

def get_dist(lat, long):
    seattle = (47.608013, -122.335167)
    house = (lat, long)
    return haversine(seattle, house, unit=Unit.MILES)

# Add distance_from_seattle column
df_clean['distance'] = df_clean.apply(lambda x: get_dist(x.lat, x.long), axis=1)

In [5]:
def get_dist_red(lat, long):
    redmond = (47.673988, -122.121513)
    house = (lat, long)
    return haversine(redmond, house, unit=Unit.MILES)

# Add distance_from_seattle column
df_clean['distance_r'] = df_clean.apply(lambda x: get_dist_red(x.lat, x.long), axis=1)

In [6]:
df_clean.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'month', 'basement',
       'renovated', 'age', 'distance', 'distance_r'],
      dtype='object')

In [7]:
def corr_check(df, threshold):
    '''
    Enter dataframe and threshold for correlation
    Returns table of the highly correlated pairs
    '''
    corr_df = df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
    corr_df['pairs'] = list(zip(corr_df.level_0, corr_df.level_1))
    corr_df.set_index(['pairs'], inplace = True)
    corr_df.drop(columns=['level_1', 'level_0'], inplace = True)
    corr_df.columns = ['cc']
    corr_df = corr_df.drop_duplicates()
    corr_df = corr_df[(corr_df['cc'] > threshold) & (corr_df['cc'] < 1)]
    return corr_df

corr_check(df_clean, .7)

,cc
pairs,
"(yr_renovated, renovated)",0.999968
"(yr_built, age)",0.999873
"(sqft_living, sqft_above)",0.876448
"(lat, distance_r)",0.861560
"(sqft_living, grade)",0.762779
"(sqft_living15, sqft_living)",0.756402
"(grade, sqft_above)",0.756073
"(sqft_living, bathrooms)",0.755758
"(sqft_living15, sqft_above)",0.731767


In [8]:
# For consistent randomness
np.random.seed(42)

## Modeling

In [9]:
lr = LinearRegression()

In [10]:
# Our model needs to have only numeric variables.
# Using this function, we can drop all columns without numeric varibales.
# We will input this function within our next function.
def only_numeric(data):
    '''returns a dataframe with only numeric values'''
    for column in data.columns:
        if is_numeric_dtype(data[column]) == False:
            data = data.drop(column, axis=1)
        else:
            continue
    return data

In [11]:
# This returns our y and X for any data frame. 
# Uses all the numeric columns, need to pass a string as a target variable.
def get_y_X(data, target):
    data = only_numeric(data) # Making data only columns with numeric values.
    y = data[target] 
    X = data.drop(target, axis=1)
    return y, X

In [12]:
# This function will return a train / test split variables for an X and y. 
def my_train_test(ys, Xs):
    X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=.2)

    return X_train, X_test, y_train, y_test

In [13]:
# This prediction function is not in effect, work in progress. 
def prediction(ys, Xs):
    y_hat = lr.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_hat))
    return rmse, y_hat

In [14]:
# Function to compare R2 values and RMSE values of the train and testing models
def train_test_compare(X_tr, X_te, y_tr, y_te):
    model = lr.fit(X_tr, y_tr) # fit the model
    
    #R2 Scores
    train_score = lr.score(X_tr, y_tr)
    test_score = lr.score(X_te, y_te)
    
    #RMSE
    y_hat_train = lr.predict(X_tr)
    y_hat_test = lr.predict(X_te)
    
    train_rmse = np.sqrt(mean_squared_error(y_tr, y_hat_train))
    test_rmse = np.sqrt(mean_squared_error(y_te, y_hat_test))
    
    #intercept / coef
    inter = model.intercept_
    stats = sm.OLS(y_tr, sm.add_constant(X_tr)).fit()
    summary = stats.summary()
    
    return print(f' training data R2: {train_score}\n testing data R2: {test_score} \
                    \n training data rmse: {train_rmse}\n testing data rmse: {test_rmse} \
                    \n {summary}') 

In [15]:
# I am not sure if I am doing the RMSE correctly, but I am pretty confident with the R2

## Model with Test Data

In [16]:
# Using test data to demonstrate
test_data = df_clean.loc[:,['price', 'bedrooms', 'condition', 'sqft_living']]

In [17]:
y, X = get_y_X(test_data, 'price')

X_train, X_test, y_train, y_test = my_train_test(y, X)

train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.5163831722955119
 testing data R2: 0.5054004287437497                     
 training data rmse: 256582.8889026345
 testing data rmse: 253781.23712101585                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     6148.
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:11   Log-Likelihood:            -2.3970e+05
No. Observations:               17277   AIC:                         4.794e+05
Df Residuals:                   17273   BIC:                         4.794e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
              

## Model on with Entire Clean DF 

In [18]:
y, X = get_y_X(df_clean, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.6948403066560066
 testing data R2: 0.7004597754601365                     
 training data rmse: 201732.9924066607
 testing data rmse: 205723.7922036507                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.695
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     1871.
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:11   Log-Likelihood:            -2.3555e+05
No. Observations:               17277   AIC:                         4.711e+05
Df Residuals:                   17255   BIC:                         4.713e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
               

## Model with Dummy Variables for Categories

In [19]:
df_clean_dumm = df_clean.copy()

In [20]:
# Get dummies
zipcode_dummies = pd.get_dummies(df_clean_dumm['zipcode'], drop_first=True)
waterfront_dummies = pd.get_dummies(df_clean_dumm['waterfront'], drop_first=True)
view_dummies = pd.get_dummies(df_clean_dumm['view'], drop_first=True)
month_dummies = pd.get_dummies(df_clean_dumm['month'], drop_first=True)

df_clean_dumm = pd.concat([df_clean_dumm, waterfront_dummies, 
                           view_dummies, month_dummies, zipcode_dummies], axis=1)

In [21]:
df_clean_dumm

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NO,NONE,...,0,0,0,0,0,0,1,0,0,0
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,1,0,0,0,0,0,0,0,0,0
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,NO,NONE,...,0,0,0,0,0,0,0,0,0,0


In [22]:
y, X = get_y_X(df_clean_dumm, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.8129368675173206
 testing data R2: 0.8142620065637962                     
 training data rmse: 159155.94225671375
 testing data rmse: 157206.82337219326                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.813
Model:                            OLS   Adj. R-squared:                  0.812
Method:                 Least Squares   F-statistic:                     717.6
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:12   Log-Likelihood:            -2.3145e+05
No. Observations:               17277   AIC:                         4.631e+05
Df Residuals:                   17172   BIC:                         4.639e+05
Df Model:                         104                                         
Covariance Type:            nonrobust                                         
             

## Model with non-Luxury houses w/ Dummy Variabls

In [23]:
non_lux = df_clean_dumm.copy()

In [24]:
non_lux = non_lux[non_lux['price'] < 1000000]
non_lux.drop(columns=['yr_built'], inplace=True)

In [25]:
y, X = get_y_X(non_lux, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.8396917350262207
 testing data R2: 0.8333477169665032                     
 training data rmse: 78340.10788685206
 testing data rmse: 79535.32890773733                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     812.7
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:14   Log-Likelihood:            -2.0408e+05
No. Observations:               16085   AIC:                         4.084e+05
Df Residuals:                   15981   BIC:                         4.092e+05
Df Model:                         103                                         
Covariance Type:            nonrobust                                         
               

In [26]:
model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.840
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     812.7
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:14   Log-Likelihood:            -2.0408e+05
No. Observations:               16085   AIC:                         4.084e+05
Df Residuals:                   15981   BIC:                         4.092e+05
Df Model:                         103                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
id            -1.664e-07   2.25e-07     -0.739      0.460   -6.08e-07    2.75e-07
bedrooms      -3216.1573    878.847     -3.660      0.000   -4938.797   -1493.518
bathrooms      1.165e+04   1543.992      7.546      0.000    8623.822    1.47e+04
sqft_living      61.4954      2.201     27.945      0.000      57.182      65.809
sqft_lot          0.3150      0.023     13.680      0.000       0.270       0.360
floors        -1.484e+04   1847.563     -8.033      0.000   -1.85e+04   -1.12e+04
condition      2.527e+04   1101.109     22.947      0.000    2.31e+04    2.74e+04
grade          4.004e+04   1074.747     37.257      0.000    3.79e+04    4.21e+04
sqft_above       47.0172      2.226     21.125      0.000      42.655      51.380
yr_renovated   1848.8248    223.837      8.260      0.000    1410.079    2287.571
zipcode         513.9742     50.776     10.122      0.000     414.448     613.500
lat            -5.25e+04   4.21e+04     -1.248      0.212   -1.35e+05    2.99e+04
long           3.921e+05   3.71e+04     10.575      0.000    3.19e+05    4.65e+05
sqft_living15    34.8052      1.810     19.235      0.000      31.258      38.352
sqft_lot15        0.0141      0.037      0.378      0.706      -0.059       0.087
month          -179.4654    392.753     -0.457      0.648    -949.305     590.374
basement       -363.1851    715.549     -0.508      0.612   -1765.741    1039.371
renovated     -3.652e+06   4.47e+05     -8.177      0.000   -4.53e+06   -2.78e+06
age             412.1458     37.693     10.934      0.000     338.263     486.029
distance      -1.301e+04   1081.804    -12.031      0.000   -1.51e+04   -1.09e+04
distance_r     1405.2620   1031.462      1.362      0.173    -616.519    3427.043
YES            1.258e+05   1.49e+04      8.469      0.000    9.67e+04    1.55e+05
EXCELLENT      9.136e+04   9330.175      9.791      0.000    7.31e+04     1.1e+05
FAIR            342.4620   6353.533      0.054      0.957   -1.21e+04    1.28e+04
GOOD           2.023e+04   5939.222      3.406      0.001    8584.909    3.19e+04
NONE          -5.004e+04   3392.691    -14.749      0.000   -5.67e+04   -4.34e+04
2              7403.2507   3627.407      2.041      0.041     293.125    1.45e+04
3              2.077e+04   3015.731      6.889      0.000    1.49e+04    2.67e+04
4              2.886e+04   2622.343     11.005      0.000    2.37e+04     3.4e+04
5              2197.3222   2321.472      0.947      0.344   -2353.024    6747.669
6              2601.4685   2194.820      1.185      0.236   -1700.625    6903.562
7             -1363.0227   2003.126     -0.680      0.496   -5289.375    2563.329
8             -2388.6449   2007.990     -1.190      0.234   -6324.531    1547.241
9             -5144.1696   2016.840     -2.551      0.011   -9097.402   -1190.937
10            -3995.5761   2006.577     -1.991   

## Model with non-Luxury and non-Cheap houses w/ Dummy Variabls

In [27]:
no_lux_cheap = non_lux.copy()

In [28]:
no_lux_cheap = no_lux_cheap[no_lux_cheap['price'] > 100000]

In [29]:
y, X = get_y_X(no_lux_cheap, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

 training data R2: 0.8390231842405416
 testing data R2: 0.8327932359356818                     
 training data rmse: 78111.96107340786
 testing data rmse: 80440.4112144408                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.838
Method:                 Least Squares   F-statistic:                     807.5
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:15   Log-Likelihood:            -2.0373e+05
No. Observations:               16061   AIC:                         4.077e+05
Df Residuals:                   15957   BIC:                         4.085e+05
Df Model:                         103                                         
Covariance Type:            nonrobust                                         
                

## Model with non-Luxury houses w/ Dummy Variables - drop recurring columns

In [30]:
non_lux_drop = non_lux.copy()
non_lux_drop = non_lux_drop.drop(['id','lat', 'long', 'sqft_lot15', 'month', 'waterfront', 'zipcode', 'view'], axis=1)

In [31]:
y, X = get_y_X(non_lux_drop, 'price')
X_train, X_test, y_train, y_test = my_train_test(y, X)
train_test_compare(X_train, X_test, y_train, y_test)

model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
model.summary()

 training data R2: 0.8376272919254353
 testing data R2: 0.8361465565991268                     
 training data rmse: 78878.42683343534
 testing data rmse: 78719.16787326234                     
                             OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     832.9
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:16   Log-Likelihood:            -2.0419e+05
No. Observations:               16085   AIC:                         4.086e+05
Df Residuals:                   15985   BIC:                         4.094e+05
Df Model:                          99                                         
Covariance Type:            nonrobust                                         
               

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     832.9
Date:                Tue, 04 Jan 2022   Prob (F-statistic):               0.00
Time:                        15:08:17   Log-Likelihood:            -2.0419e+05
No. Observations:               16085   AIC:                         4.086e+05
Df Residuals:                   15985   BIC:                         4.094e+05
Df Model:                          99                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
bedrooms      -4515.8649    944.579     -4.781      0.000   -6367.346   -2664.384
bathrooms      1.273e+04   1565.320      8.134      0.000    9664.547    1.58e+04
sqft_living      61.6547      2.205     27.965      0.000      57.333      65.976
sqft_lot          0.2922      0.018     16.592      0.000       0.258       0.327
floors        -1.534e+04   1855.528     -8.270      0.000    -1.9e+04   -1.17e+04
condition      2.501e+04   1107.622     22.576      0.000    2.28e+04    2.72e+04
grade          4.051e+04   1082.449     37.423      0.000    3.84e+04    4.26e+04
sqft_above       46.9322      2.245     20.904      0.000      42.531      51.333
yr_renovated   1806.4118    225.789      8.000      0.000    1363.841    2248.983
sqft_living15    34.9323      1.815     19.250      0.000      31.375      38.489
basement      -1.523e+05    1.7e+04     -8.972      0.000   -1.86e+05   -1.19e+05
renovated     -3.577e+06   4.51e+05     -7.939      0.000   -4.46e+06   -2.69e+06
age             425.1720     37.699     11.278      0.000     351.278     499.066
distance      -6313.1275    822.555     -7.675      0.000   -7925.428   -4700.827
distance_r     -670.8818    872.901     -0.769      0.442   -2381.866    1040.103
YES            1.472e+05   1.51e+04      9.719      0.000    1.18e+05    1.77e+05
EXCELLENT      9.296e+04   9458.755      9.828      0.000    7.44e+04    1.11e+05
FAIR           2622.0650   6393.187      0.410      0.682   -9909.300    1.52e+04
GOOD           2.621e+04   5894.964      4.445      0.000    1.47e+04    3.78e+04
NONE          -4.945e+04   3411.518    -14.496      0.000   -5.61e+04   -4.28e+04
2              9247.2699   3916.784      2.361      0.018    1569.932    1.69e+04
3              2.183e+04   3627.611      6.018      0.000    1.47e+04    2.89e+04
4              3.023e+04   3542.203      8.535      0.000    2.33e+04    3.72e+04
5              5336.8583   3496.295      1.526      0.127   -1516.272    1.22e+04
6              5134.5568   3542.713      1.449      0.147   -1809.559    1.21e+04
7              -955.5749   3542.358     -0.270      0.787   -7898.995    5987.846
8               181.3116   3600.242      0.050      0.960   -6875.568    7238.191
9             -4950.4310   3657.391     -1.354      0.176   -1.21e+04    2218.466
10            -1404.9388   3644.337     -0.386      0.700   -8548.249    5738.372
11            -2480.7652   3857.461     -0.643      0.520      -1e+04    5080.293
12            -3905.0410   3778.792     -1.033      0.301   -1.13e+04    3501.816
98002          7849.1151   8014.992      0.979      0.327   -7861.169    2.36e+04
98003         -7212.0704   7088.770     -1.017      0.309   -2.11e+04    6682.716
98004          3.864e+05   1.38e+04     28.007      0.000    3.59e+05    4.13e+05
98005          2.261e+05   1.43e+04     15.799   

## Visualize Train vs. Test

In [32]:
# I want to make another function to visualize both the training and test, not sure if possible

In [33]:
# plt.scatter(X_test, y_test, color="black")
# plt.plot(X_test, y_pred, color="blue", linewidth=3);

# Other

In [34]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21597 non-null  int64         
 1   date           21597 non-null  datetime64[ns]
 2   price          21597 non-null  float64       
 3   bedrooms       21597 non-null  int64         
 4   bathrooms      21597 non-null  float64       
 5   sqft_living    21597 non-null  int64         
 6   sqft_lot       21597 non-null  int64         
 7   floors         21597 non-null  float64       
 8   waterfront     21597 non-null  object        
 9   view           21597 non-null  object        
 10  condition      21597 non-null  int64         
 11  grade          21597 non-null  int64         
 12  sqft_above     21597 non-null  int64         
 13  sqft_basement  21597 non-null  object        
 14  yr_built       21597 non-null  int64         
 15  yr_renovated   2159

In [35]:
df_clean.corr().abs()['price'].sort_values()

high_corr_cols = ['sqft_living', 'sqft_above', 'sqft_living15', 'bathrooms', 'sqft_basement', 'bedrooms']

In [36]:
y = df_clean['price']
X = df_clean
    
reg = LinearRegression().fit(X, y)

plt.scatter(X, y, color='green')
plt.plot(X, reg.predict(X))
plt.xlabel('sqft_living')
plt.ylabel('Price');

TypeError: invalid type promotion

In [ ]:
for x in high_corr_cols:
    y = df_clean['price']
    X = df_clean[x]
    
    reg = LinearRegression().fit(X, y)

    plt.scatter(X, y, color='green')
    plt.plot(X, reg.predict(X))
    plt.xlabel(x)
    plt.ylabel('Price');